In [1]:
import importlib
cim_profile = "rc4_2021"
cim = importlib.import_module("cimgraph.data_profile." + cim_profile)

In [2]:
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import RDFlibConnection
from cimgraph.models import NodeBreakerModel
import json

In [3]:
# RDFLib File Reader Connection
params = ConnectionParameters(filename="../cimgraph/tests/test_models/maple10nodebreaker.xml", cim_profile=cim_profile, iec61970_301=7)
rdf = RDFlibConnection(params)

In [4]:
geo_region_id = "_EE4C60AE-550D-4599-92F4-022DF3118B3C"
geo_region = cim.GeographicalRegion(mRID = geo_region_id)

In [5]:
network = NodeBreakerModel(connection=rdf, container=geo_region, distributed=False)

In [6]:
network = NodeBreakerModel(connection=rdf, container=geo_region, distributed=True)

In [20]:
for sr_area in network.distributed_areas[cim.SubGeographicalRegion].values():
    print("subregion", sr_area.container.name)
    for sub_area in sr_area.distributed_areas[cim.Substation].values():
        print("substation", sub_area.container.name)

        for vl_area in sub_area.distributed_areas[cim.VoltageLevel].values():
            print("voltage level", vl_area.container.name)
            
        for feeder_area in sub_area.distributed_areas[cim.Feeder].values():
            print("feeder", feeder_area.container.name, "contains PV aggregates")
            feeder_area.get_all_edges(cim.PowerElectronicsConnection)
            feeder_area.get_all_edges(cim.PhotovoltaicUnit)
            if cim.PowerElectronicsConnection in feeder_area.graph:
                for pv in feeder_area.graph[cim.PowerElectronicsConnection].values():
                    print(pv.name, float(pv.p)/1000000, "MW")
            else:
                print("none")


subregion small
substation maple10bus_sub2
voltage level SUB2_115.0_B1
voltage level SUB2_4.16_B1
voltage level SUB2_34.5_B1
voltage level SUB2_230.0_B1
feeder feeder_11 contains PV aggregates
s10_der_ag2 1.0 MW
feeder feeder_9 contains PV aggregates
s9_der_ag2 2.0 MW
feeder feeder_8 contains PV aggregates
s9_der_ag1 1.0 MW
feeder feeder_10 contains PV aggregates
s10_der_ag1 5.0 MW
substation maple10bus_sub1
voltage level SUB1_230.0_B1
voltage level SUB1_115.0_B1
voltage level SUB1_12.47_B1
voltage level SUB1_34.5_B1
feeder feeder_4 contains PV aggregates
s7_der_ag4 1.0 MW
feeder feeder_1 contains PV aggregates
s7_der_ag1 2.0 MW
feeder feeder_5 contains PV aggregates
s8_der_ag1 5.0 MW
feeder feeder_6 contains PV aggregates
s8_der_ag2 2.0 MW
feeder feeder_3 contains PV aggregates
s7_der_ag3 2.0 MW
feeder feeder_7 contains PV aggregates
s8_der_ag3 6.0 MW
feeder feeder_2 contains PV aggregates
none
